# Regressão Linear 

## MQO 

In [2]:
# Inicializando Spark
import findspark
findspark.init('/home/macaubas/spark-3.2.1-bin-hadoop3.2')
import pyspark

from pyspark.sql import SparkSession 

spark = SparkSession.builder.appName("linearReg").getOrCreate()

22/05/16 17:28:22 WARN Utils: Your hostname, macaubas-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/05/16 17:28:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/16 17:28:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/16 17:28:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/16 17:28:24 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


### Imports

In [3]:
# Regressão linear
from pyspark.ml.regression import LinearRegression

# Vetores
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler


### Carregando os dados

In [4]:
# Load data
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)

data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



**Alvo da predição: Yearly Amount Spent**

In [5]:
data.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

### Criando vetores

In [6]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

**Coluna Label: Yearly Amount Spent**

**Features: Avg Session Length, Time on App, Time on Website, Length of Membership**

In [7]:
# Criando as features (características)
assembler = VectorAssembler(inputCols = ['Avg Session Length',
                                         'Time on App',
                                         'Time on Website',
                                         'Length of Membership'],
                           outputCol='features')


# Transformando os dados
output = assembler.transform(data)

output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [8]:
#Visualizando Features
output.select('Features').head(1)

[Row(Features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [9]:
# Selecionando Dados finais
final_data = output.select('features', 'Yearly Amount Spent')

# Separando entre test e treino
train_data, test_data  = final_data.randomSplit([0.7,0.3])

# Visualizando Treino
train_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[29.5324289670579...|  408.6403510726275|
|[30.4925366965402...|  282.4712457199145|
|[30.5743636841713...| 442.06441375806565|
|[30.7377203726281...|  461.7807421962299|
|[30.8364326747734...|  467.5019004269896|
|[30.8794843441274...|  490.2065999848547|
|[31.0472221394875...|  392.4973991890214|
|[31.0613251567161...|  487.5554580579016|
|[31.0662181616375...| 448.93329320767435|
|[31.1239743499119...|  486.9470538397658|
|[31.2681042107507...|  423.4705331738239|
|[31.2834474760581...|  591.7810894256675|
|[31.3662121671876...|  430.5888825564849|
|[31.3895854806643...|  410.0696110599829|
|[31.4252268808548...|  530.7667186547619|
|[31.4459724827577...| 484.87696493512857|
|[31.4474464941278...|   418.602742095224|
|[31.5257524169682...|  443.9656268098819|
|[31.5261978982398...|  409.0945261923378|
|[31.5316044825729...| 436.51560572936256|
+----------

### Descrevendo treino e teste

In [10]:
# Treino
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                371|
|   mean| 498.44218724779665|
| stddev|   80.4365993550531|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [11]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                129|
|   mean| 501.82145473149524|
| stddev|  76.24570653513717|
|    min|   266.086340948469|
|    max|  712.3963268096637|
+-------+-------------------+



### Modelo

In [12]:
# Caracterizando o modelo
lr = LinearRegression(labelCol= 'Yearly Amount Spent')

# Fitting model aos dados treinados
lr_Model = lr.fit(train_data)

22/05/16 17:28:36 WARN Instrumentation: [495a13cb] regParam is zero, which might cause numerical instability and overfitting.
22/05/16 17:28:36 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/05/16 17:28:36 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/05/16 17:28:36 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [13]:
# Coeficientes
lr_Model.coefficients

DenseVector([25.8379, 38.5926, 0.304, 61.2026])

In [14]:
# Intercepto
lr_Model.intercept

-1047.1294360114869

### Avaliando o modelo

In [15]:
# Checando ajuste
test_results = lr_Model.evaluate(test_data)

In [16]:
# Residuals
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-12.320928886310298|
| -17.99151286538256|
|  6.909088321016043|
| -6.865788178556386|
|  9.579149907186093|
|-1.2535191464459103|
|  2.624642445875338|
| 18.973997006305183|
| 3.9531909440153186|
| -5.087349809598891|
| -5.213880513291372|
|-17.390715757553153|
| -13.99198928312876|
| -6.451853198585127|
|  2.429155732480069|
| -4.272659109258939|
|  7.226152160498657|
|-2.5771052536251773|
| -6.179210799073132|
| -2.611593723638805|
+-------------------+
only showing top 20 rows



/home/macaubas/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [17]:
# R2
test_results.r2

0.9842389491069861

In [18]:
# RMSE
test_results.rootMeanSquaredError

9.53494316222058

### Deploying model - apenas features


Baseados nas características, qual a previsão? 

In [19]:
unlabeled_data = test_data.select('features')

unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.3931845423455...|
|[30.8162006488763...|
|[30.9716756438877...|
|[31.1280900496166...|
|[31.1695067987115...|
|[31.2606468698795...|
|[31.3091926408918...|
|[31.3123495994443...|
|[31.3584771924370...|
|[31.5147378578019...|
|[31.5171218025062...|
|[31.5702008293202...|
|[31.5741380228732...|
|[31.7207699002873...|
|[31.7366356860502...|
|[31.8124825597242...|
|[31.8209982016720...|
|[31.8530748017465...|
|[31.8745516945853...|
|[31.9120759292006...|
+--------------------+
only showing top 20 rows



In [20]:
predictions = lr_Model.transform(unlabeled_data)

predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...| 332.2497986895039|
|[30.8162006488763...|284.07785381385156|
|[30.9716756438877...| 487.7295214358767|
|[31.1280900496166...|  564.118474925611|
|[31.1695067987115...| 417.7773808951067|
|[31.2606468698795...| 422.5801504033973|
|[31.3091926408918...| 430.0960753940583|
|[31.3123495994443...|444.61742102163544|
|[31.3584771924370...| 491.2227595054601|
|[31.5147378578019...| 494.8998378060603|
|[31.5171218025062...| 281.1323011636771|
|[31.5702008293202...|  563.336207898958|
|[31.5741380228732...| 558.4012614437156|
|[31.7207699002873...| 545.2267866766081|
|[31.7366356860502...|494.50429052305185|
|[31.8124825597242...|397.08300409305616|
|[31.8209982016720...| 417.4491288527147|
|[31.8530748017465...|461.86222871597715|
|[31.8745516945853...| 398.4644550453406|
|[31.9120759292006...| 390.1463100293465|
+--------------------+------------